In [1]:
award_names = ['Best Motion Picture – Drama', 'Best Motion Picture – Musical or Comedy', 'Best Motion Picture – Foreign Language',
 'Best Motion Picture – Animated', 'Best Director – Motion Picture', 'Best Actor in a Motion Picture – Drama',
 'Best Actor in a Motion Picture – Musical or Comedy', 'Best Actress in a Motion Picture – Drama',
 'Best Actress in a Motion Picture – Musical or Comedy', 'Best Supporting Actor – Motion Picture',
 'Best Supporting Actress – Motion Picture', 'Best Screenplay – Motion Picture', 'Best Original Score – Motion Picture',
 'Best Original Song – Motion Picture', 'Best Television Series – Drama', 'Best Television Series – Musical or Comedy',
 'Best Miniseries or Motion Picture – Television', 'Best Actor in a Television Series – Drama',
 'Best Actor in a Television Series – Musical or Comedy', 'Best Actor in a Miniseries or Motion Picture – Television',
 'Best Actress in a Television Series – Drama', 'Best Actress in a Television Series – Musical or Comedy',
 'Best Actress in a Miniseries or Motion Picture – Television',
 'Best Supporting Actor in a Series, Miniseries or Motion Picture – Television',
 'Best Supporting Actress in a Series, Miniseries or Motion Picture – Television']
new_award_names = [0]*len(award_names)
for i in range(len(award_names)):
    new_award_names[i] = award_names[i].lower()


In [2]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict

In [3]:
df = pd.read_json('../gg2015.json')
df = df['text']
df = df[df.str.contains('won best|wins best|goes to', case = False)]
df = df[~df.str.contains('think|thinking|should|maybe', case = False)]
wins_df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)
if wins_df.size > 5000:
    wins_df = wins_df.sample(5000)

In [4]:
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [5]:
pattern1 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
pattern2 = [{"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
pattern3 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]
pattern4 = [{"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]
pattern5 = [{"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'},{"TEXT": "goes"}, {"TEXT": "to"}, {"ENT_TYPE": "PERSON", 'OP':'+'}] 
pattern6 = [{"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}, {"TEXT": "goes"}, {"TEXT": "to"}, {"ENT_TYPE": "PERSON", 'OP':'+'}] 

In [6]:
matcher = Matcher(nlp.vocab)
matcher.add("Winner", [pattern1, pattern2, pattern3, pattern4, pattern5, pattern6])

In [7]:
winners = {}
winners = defaultdict(lambda: 1, winners)
for i, text in wins_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        winners[str(span).strip(punctuation).strip()] = winners[str(span).strip(punctuation).strip()] +1

In [8]:
winners = sorted(winners.items(), key=lambda item: item[1], reverse = True)
print(winners)

[('Matt Bomer wins Best Supporting Actor', 35), ('Maggie Gyllenhaal wins Best Actress', 24), ('J.K. Simmons wins Best Supporting Actor', 18), ('Ruth Wilson wins Best Actress', 18), ('Julianne Moore Wins Best Actress', 16), ('Julianne Moore wins Best Actress', 16), ('Best Director - Motion Picture goes to Richard Linklater', 11), ('Best Screenplay - Motion Picture goes to Birdman', 10), ('J.K. Simmons Wins Best Supporting Actor', 9), ('Kevin Spacey wins Best Actor TV Drama', 9), ('Amy Adams wins Best Actress, Motion Picture Comedy', 8), ('Gina Rodriguez wins Best Actress - TV Series', 8), ('Julianne Moore wins Best Actress Drama for', 7), ('Boyhood won Best Picture', 7), ('Amy Adams wins Best Actress in a Motion Picture', 7), ('Eddie Redmayne wins Best Actor Drama', 7), ('Boyhood won Best Picture - Drama at', 6), ('Simmons wins Best Supporting Actor', 6), ('Patricia Arquette wins Best Supporting Actress for Boyhood', 6), ('Billy Bob Thornton wins Best Actor - TV Miniseries', 6), ('Birdm

In [9]:
import Levenshtein
# tweet = string, awards = dict
def DistMeasure(tweet):
    awards = {}
    tweet = tweet.lower()
    tweet = tweet.replace('tv', 'television')
    tweet = tweet.replace('film', 'motion picture')
    tweet = tweet.replace('movie', 'motion picture')
    for award in new_award_names:
        split_award = award.split()
        award_dist = 0
        
        split_tweet = tweet.split()
        for word in split_tweet:
            array = []
            for word2 in split_award:
                array.append(Levenshtein.distance(word, word2))
            min_val = array[np.argmin(array)]
            award_dist += min_val
        awards[award] = award_dist
        
    val = min(awards.values())
    res = [key for key in awards if awards[key] == val]
    if len(res) > 10:
        res = []
    return res

In [10]:
award_list = [dict() for x in range(len(new_award_names))]

for tweet in winners: 
    tweet = tweet[0]
    split = tweet.lower().split()
    
    index = split.index("best")

    award_phrase = " ".join(split[index:])
    matching_awards = DistMeasure(award_phrase)
    for a in matching_awards:
        award_index = new_award_names.index(a)
        global person
        person = ''
        if "goes" in split and "to" in split:
            print("goes")
            person_index = split.index("goes")
            person = split[person_index+2:]
            print(person)
        else:
            person = split[:index-1]
        person = " ".join(person)
        if person not in award_list[award_index]:
            award_list[award_index][person] = 1
        else:
            award_list[award_index][person] = award_list[award_index][person]+1

print(award_list)

goes
['richard', 'linklater']
goes
['birdman']
goes
['selma']
goes
['@annakfaris']
goes
['@annakfaris']
goes
['@annakfaris']
goes
['@annakfaris']
goes
['@annakfaris']
goes
['@annakfaris']
goes
['@annakfaris']
goes
['@annakfaris']
goes
['@annakfaris']
goes
['@annakfaris']
goes
['patricia', 'arquette']
goes
['lupita']
goes
['richard', 'linklater']
goes
['ruth', 'wilson']
goes
['billy', 'bob', 'thornton']
goes
['billy', 'bob', 'thornton']
goes
['billy', 'bob', 'thornton']
goes
['billy', 'bob', 'thornton']
goes
['billy', 'bob', 'thornton']
goes
['billy', 'bob', 'thornton']
goes
['billy', 'bob', 'thornton']
goes
['billy', 'bob', 'thornton']
goes
['billy', 'bob', 'thornton']
goes
['billy', 'bob', 'thornton']
goes
['julianne', 'moore']
goes
['birdman']
goes
['birdman']
goes
['selma']
goes
['johann', 'johannsson']
goes
['diane', 'kruger', '&amp']
goes
['diane', 'kruger', '&amp']
goes
['diane', 'kruger', '&amp']
goes
['diane', 'kruger', '&amp']
goes
['jóhann', 'jóhannsson']
goes
['j.k.', 'simmo

In [11]:
import operator
i = 0
awards = [[] for x in range(len(new_award_names))]
for award in award_list:
    if award != {}:
        max_val = max(award.items(), key=operator.itemgetter(1))[1]
        j = 0
        list_award = list(award)
        for b in award:
            if award[b] == max_val and len(awards[i]) < 1:
                awards[i].append(list_award[j])
            j+=1
    i+=1
print(awards)

[['boyhood'], ['billy bob thornton'], ['leviathan'], ['howtotrainyourdragon2'], ['richard linklater'], ['eddie redmayne'], ['eddie redmayne'], ['julianne moore'], ['amy adams'], ['j.k. simmons'], ['julianne moore'], ['birdman'], ['johann johannsson'], ['selma'], ['transparent_tv'], ['transparent_tv'], ['@annakfaris'], ['eddie redmayne'], ['eddie redmayne'], ['billy bob thornton'], ['ruth wilson'], ['amy adams'], ['julianne moore'], ['j.k. simmons'], ['patricia arquette']]


In [16]:

for award in awards: 
    if len(award) == 0:
        award.append("NO WINNER FOUND")
        
print(awards)

[['boyhood'], ['NO WINNER FOUND'], ['leviathan'], ['howtotrainyourdragon2'], ['NO WINNER FOUND'], ['eddie redmayne'], ['NO WINNER FOUND'], ['julianne moore'], ['amy adams'], ['j.k. simmons'], ['julianne moore'], ['birdman'], ['johann johannsson'], ['selma'], ['transparent_tv'], ['transparent_tv'], ['@annakfaris'], ['eddie redmayne'], ['eddie redmayne'], ['billy bob thornton'], ['ruth wilson'], ['amy adams'], ['julianne moore'], ['j.k. simmons'], ['patricia arquette']]


In [17]:
award_flat = np.array(awards).flatten()
print(award_flat)

['boyhood' 'NO WINNER FOUND' 'leviathan' 'howtotrainyourdragon2'
 'NO WINNER FOUND' 'eddie redmayne' 'NO WINNER FOUND' 'julianne moore'
 'amy adams' 'j.k. simmons' 'julianne moore' 'birdman' 'johann johannsson'
 'selma' 'transparent_tv' 'transparent_tv' '@annakfaris' 'eddie redmayne'
 'eddie redmayne' 'billy bob thornton' 'ruth wilson' 'amy adams'
 'julianne moore' 'j.k. simmons' 'patricia arquette']


In [18]:
result = dict(zip(award_names, award_flat)) 
for res in result:
    print(res, result[res])

Best Motion Picture – Drama boyhood
Best Motion Picture – Musical or Comedy NO WINNER FOUND
Best Motion Picture – Foreign Language leviathan
Best Motion Picture – Animated howtotrainyourdragon2
Best Director – Motion Picture NO WINNER FOUND
Best Actor in a Motion Picture – Drama eddie redmayne
Best Actor in a Motion Picture – Musical or Comedy NO WINNER FOUND
Best Actress in a Motion Picture – Drama julianne moore
Best Actress in a Motion Picture – Musical or Comedy amy adams
Best Supporting Actor – Motion Picture j.k. simmons
Best Supporting Actress – Motion Picture julianne moore
Best Screenplay – Motion Picture birdman
Best Original Score – Motion Picture johann johannsson
Best Original Song – Motion Picture selma
Best Television Series – Drama transparent_tv
Best Television Series – Musical or Comedy transparent_tv
Best Miniseries or Motion Picture – Television @annakfaris
Best Actor in a Television Series – Drama eddie redmayne
Best Actor in a Television Series – Musical or Comedy